In [1]:
# hdfs name node information
#http://localhost:50070/dfshealth.html#tab-overview
# http://localhost:8088/cluster # resouce manager hadoop,does not show spark at moment
# running spark application http://10.0.2.15:4041
# http://10.0.2.15:4040/jobs/ ****
#history server http://10.0.2.15:18080 not working?

#sc = SparkContext.getOrCreate()
#from pyspark import SparkConf, SparkContext

import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, HiveContext
import os
from pyspark.sql.functions import *
import pyspark.storagelevel 
import numpy as np
import pandas as pd
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pdb
import subprocess # Used for executing linux commands, for writing to teradata
import sys

from pyspark.sql.types import *
from pyspark.sql.functions import col, udf, min, max
from datetime import datetime, timedelta

conf = SparkConf().setMaster("local").setAppName("TamingBigData")
sc = SparkContext.getOrCreate(conf = conf)
sqlContext = SQLContext(sc)
sqlContext = HiveContext(sc)

In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("MinTemperatures")
sc = SparkContext.getOrCreate(conf = conf)

In [2]:
#conf = SparkConf().setMaster("local").setAppName("MinTemperatures")
#sc = SparkContext.getOrCreate()

In [3]:
import collections
conf = SparkConf().setAppName("RatingsHistogram")
lines =  sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/ml-100k")
#lines.take(2)
ratings = lines.map(lambda x: x.split()[2]) # take 3rd column only
result =  ratings.countByValue()
print(result,":",type(result))
sortedResults = collections.OrderedDict(sorted(result.items()))
print(sortedResults,":",type(sortedResults))
for key,value in sortedResults.items():
    print("%s,%s" %(key,value))
#by parsing data into different fields

(defaultdict(<type 'int'>, {u'1': 6110, u'3': 27145, u'2': 11370, u'5': 21201, u'4': 34174}), ':', <type 'collections.defaultdict'>)
(OrderedDict([(u'1', 6110), (u'2', 11370), (u'3', 27145), (u'4', 34174), (u'5', 21201)]), ':', <class 'collections.OrderedDict'>)
1,6110
2,11370
3,27145
4,34174
5,21201


In [59]:
dct = {"bob":2,"map":4,"nap":3}
type(dct)

dict

In [64]:
# to create a small rdd we use parallize other wise sc.textFile or HiveContext.sql(query)

In [116]:
rdd = sc.parallelize([1,2,3,4])
rdd2 = sc.parallelize([5,2,1,4])

newrdd = rdd.map(lambda x:x*2) # map takes each element in rdd ,processes it 
#generates a new rdd with same number of elements

In [83]:
print(rdd.take(4))
print(newrdd.take(4))

[1, 2, 3, 4]
[2, 4, 6, 8]


In [84]:
def doubleit(x): # same thig can be achived using def
    return x*2

In [85]:
newrdd=rdd.map(doubleit)
print(newrdd.take(4))

[2, 4, 6, 8]


In [110]:
#RDD ACTIONS
# take,collect,count,countByValue,top,reduce

In [126]:
print(newrdd.collect())
print(newrdd.count())
print(newrdd.top(2)) # top2
print(newrdd.sortBy(lambda x:x,ascending=True).take(2)) # bottom2
print(newrdd.min())
print(newrdd.countByValue())

#print(newrdd.reduce(lambda x+x:x))

[2, 4, 6, 8]
4
[8, 6]
[2, 4]
2
defaultdict(<type 'int'>, {8: 1, 2: 1, 4: 1, 6: 1})


In [36]:
# simple word count
wrdRdd = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/wrdCnt.txt")
print(wrdRdd.collect())
spltRdd= wrdRdd.flatMap(lambda x:x.split())
cntvalue = spltRdd.countByValue()# alternative way of counting,using dict
print(type(cntvalue),cntvalue) # alternative way of counting,using dict
print(spltRdd.collect())
mapRdd=spltRdd.map(lambda x: (x,1)) # create key value pair
print(mapRdd.collect())
mapRdd.reduceByKey(lambda x,y:x+y).collect() #add the values of the same keys, x & y are2 values of same key
# this is now like a huge nosql db

[u'how are you guys doing . you guys are so sick you guys cannot be doing it', u'you guys are impossible']
(<type 'collections.defaultdict'>, defaultdict(<type 'int'>, {u'be': 1, u'doing': 2, u'it': 1, u'.': 1, u'how': 1, u'cannot': 1, u'so': 1, u'are': 3, u'sick': 1, u'impossible': 1, u'you': 4, u'guys': 4}))
[u'how', u'are', u'you', u'guys', u'doing', u'.', u'you', u'guys', u'are', u'so', u'sick', u'you', u'guys', u'cannot', u'be', u'doing', u'it', u'you', u'guys', u'are', u'impossible']
[(u'how', 1), (u'are', 1), (u'you', 1), (u'guys', 1), (u'doing', 1), (u'.', 1), (u'you', 1), (u'guys', 1), (u'are', 1), (u'so', 1), (u'sick', 1), (u'you', 1), (u'guys', 1), (u'cannot', 1), (u'be', 1), (u'doing', 1), (u'it', 1), (u'you', 1), (u'guys', 1), (u'are', 1), (u'impossible', 1)]


[(u'be', 1),
 (u'doing', 2),
 (u'it', 1),
 (u'.', 1),
 (u'how', 1),
 (u'cannot', 1),
 (u'so', 1),
 (u'are', 3),
 (u'sick', 1),
 (u'impossible', 1),
 (u'you', 4),
 (u'guys', 4)]

In [32]:
# map
print(wrdRdd.count(),wrdRdd.collect())
print("------------------------")
maprdd = wrdRdd.map(lambda x:x.split())
print(maprdd.count(),maprdd.collect())
print("------------------------")

# flatmap
fltmaprdd = wrdRdd.flatMap(lambda x:x.split())
print(fltmaprdd.count(),fltmaprdd.collect())
print("------------------------")


(2, [u'how are you guys doing . you guys are so sick you guys cannot be doing it', u'you guys are impossible'])
------------------------
(2, [[u'how', u'are', u'you', u'guys', u'doing', u'.', u'you', u'guys', u'are', u'so', u'sick', u'you', u'guys', u'cannot', u'be', u'doing', u'it'], [u'you', u'guys', u'are', u'impossible']])
------------------------
(21, [u'how', u'are', u'you', u'guys', u'doing', u'.', u'you', u'guys', u'are', u'so', u'sick', u'you', u'guys', u'cannot', u'be', u'doing', u'it', u'you', u'guys', u'are', u'impossible'])
------------------------


mapValues is only applicable for PairRDDs, meaning RDDs of the form RDD[(A, B)]. In that case, mapValues operates on the value only (the second part of the tuple), while map operates on the entire record (tuple of key and value).

In [271]:
# using mapValues and reduceKeys # rcap of what I learnt below
lines = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/fakefriendstmp.csv")
print("Entire data set : ",lines.collect())
#I need the 2nd and 3rd values only- of each line
print(lines.map(lambda x: x.split(",")).collect())
comaSplit = lines.map(lambda x: x.split(","))
rdd = lines.map(parseLine) # parseLine is regular python parser
print(rdd.collect())

# create key and values
kv = rdd.mapValues(lambda x:(x,1))
print(kv.collect())
# reduce the kv 
red =  kv.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])) # (33, (10, 1)) , (33, (15, 1))
print(red.collect())
# find average , average of each value ,which is one to one operation
avg = red.mapValues(lambda x: x[0]/x[1])
print("Average friends:",avg.collect())


('Entire data set : ', [u'0,Will,33,10', u'1,Jean-Luc,26,2', u'2,Hugh,55,12', u'3,Deanna,55,10', u'4,Quark,68,14', u'5,Weyoun,33,15', u'6,Gowron,55,20', u'7,Will,68,15', u'8,Jadzia,26,5', u'9,Hugh,26,20', u'10,Odo,55,11'])
[[u'0', u'Will', u'33', u'10'], [u'1', u'Jean-Luc', u'26', u'2'], [u'2', u'Hugh', u'55', u'12'], [u'3', u'Deanna', u'55', u'10'], [u'4', u'Quark', u'68', u'14'], [u'5', u'Weyoun', u'33', u'15'], [u'6', u'Gowron', u'55', u'20'], [u'7', u'Will', u'68', u'15'], [u'8', u'Jadzia', u'26', u'5'], [u'9', u'Hugh', u'26', u'20'], [u'10', u'Odo', u'55', u'11']]
[(33, 10), (26, 2), (55, 12), (55, 10), (68, 14), (33, 15), (55, 20), (68, 15), (26, 5), (26, 20), (55, 11)]
[(33, (10, 1)), (26, (2, 1)), (55, (12, 1)), (55, (10, 1)), (68, (14, 1)), (33, (15, 1)), (55, (20, 1)), (68, (15, 1)), (26, (5, 1)), (26, (20, 1)), (55, (11, 1))]
[(33, (25, 2)), (26, (27, 3)), (68, (29, 2)), (55, (53, 4))]
('Average friends:', [(33, 12), (26, 9), (68, 14), (55, 13)])


In [208]:
#FRIENDS BY AGE EXAMPLE
# WHAT IS THE AVERAGE NUMBER OF FRIENDS FOR A AVERAGE 33 YEAR OLD

In [236]:
lines = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/fakefriends.csv")
# we have a lines rdd, create a function to parseLine and get fields you want
def parseLine(line): # this funcation is regular python function and takes a line
    fields = line.split(',')
    age = int(fields[2])
    numFiends = int(fields[3])
    return (age,numFiends)

rdd = lines.map(parseLine)
print (type(rdd),rdd.take(5))
       

totalByAge = rdd.mapValues(lambda x: (x,1)).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])) 
averageByAge = totalByAge.mapValues(lambda x:x[0]/x[1])
print("Average friends by age",averageByAge.take(5))
# below is explanation

(<class 'pyspark.rdd.PipelinedRDD'>, [(33, 385), (26, 2), (55, 221), (40, 465), (68, 21)])
('Average friends by age', [(18, 343), (19, 213), (20, 165), (21, 350), (22, 206)])


In [237]:
totalByAge = rdd.mapValues(lambda x: (x,1)) # adds 1 next to values and keepys key same
#[(33, (385, 1)), (26, (2, 1)), (55, (221, 1))]

In [238]:
totalByAge.take(3)

[(33, (385, 1)), (26, (2, 1)), (55, (221, 1))]

In [239]:
totalByAge = rdd.mapValues(lambda x: (x,1)).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])) 
# reduceByKey take 33 and sum the values both 0(385) and 1 position (1),all we do
#is combine things with same key together
# x& y are i and i+1****

In [240]:
totalByAge.take(3)

[(18, (2747, 8)), (19, (2346, 11)), (20, (825, 5))]

In [241]:
averageByAge = totalByAge.mapValues(lambda x:x[0]/x[1])

In [273]:
averageByAge.take(3)

[(18, 343), (19, 213), (20, 165)]

In [2]:
# FILTERING RDD AND MINUMUM TEMPERATURE BY LOCATION
# FIND THE MINIMUM TEMPERATURE OBSERVED AT VARIOUS STATIONS IN ANY GIVEN YEAR

In [4]:
lines = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/1800.csv")

In [5]:
lines.count()

1825

In [6]:
lines.take(2)

[u'ITE00100554,18000101,TMAX,-75,,,E,', u'ITE00100554,18000101,TMIN,-148,,,E,']

In [7]:
# write a parse def: obj delimate by "," & get stationID,entryType,temperature

In [8]:
def parseLine(line):
    fields = line.split(",")
    stationId = fields[0]
    entryType = fields[2]
    temperature = float(fields[3])*0.1*(9.0/5.0)+32.0
    return (stationId,entryType,temperature)
    

In [9]:
parsedLines = lines.map(parseLine)
lines.map(parseLine).take(5)

[(u'ITE00100554', u'TMAX', 18.5),
 (u'ITE00100554', u'TMIN', 5.359999999999999),
 (u'GM000010962', u'PRCP', 32.0),
 (u'EZE00100082', u'TMAX', 16.52),
 (u'EZE00100082', u'TMIN', 7.699999999999999)]

In [10]:
minTemps = parsedLines.filter(lambda x: "TMIN"  in x[1])
print(minTemps.count())
minTemps.take(5)

730


[(u'ITE00100554', u'TMIN', 5.359999999999999),
 (u'EZE00100082', u'TMIN', 7.699999999999999),
 (u'ITE00100554', u'TMIN', 9.5),
 (u'EZE00100082', u'TMIN', 8.599999999999998),
 (u'ITE00100554', u'TMIN', 23.72)]

In [11]:
#create stationId & temperature key value pairs

In [12]:
stationTemps = minTemps.map(lambda x : (x[0],x[2])) # (x[0] key,x[2] value)
stationTemps.take(5)

[(u'ITE00100554', 5.359999999999999),
 (u'EZE00100082', 7.699999999999999),
 (u'ITE00100554', 9.5),
 (u'EZE00100082', 8.599999999999998),
 (u'ITE00100554', 23.72)]

In [13]:
# find minimum temperature by station ID
minTemps = stationTemps.reduceByKey(lambda x,y: min(x,y))

In [20]:
minTemps.take(5)  # works on spark-submit spark-submit min-temperatures.py

[(u'ITE00100554', 5.359999999999999), (u'EZE00100082', 7.699999999999999)]

In [22]:
results = minTemps.collect()  # worked after changing sc as in line 3 , was getting a stage error before that
print(type(results)) # you do a collect on rdd to get a list
for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

<type 'list'>
ITE00100554	5.36F
EZE00100082	7.70F


from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("MinTemperatures")
sc = SparkContext.getOrCreate(conf = conf)

def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

#lines = sc.textFile("file:///SparkCourse/1800.csv")
lines = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/1800.csv")
parsedLines = lines.map(parseLine)
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: min(x,y))
results = minTemps.collect();

for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

In [23]:
# FIND MAX TEMPERATURE
def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

#lines = sc.textFile("file:///SparkCourse/1800.csv")
lines = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/1800.csv")
parsedLines = lines.map(parseLine)
minTemps = parsedLines.filter(lambda x: "TMAX" in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: max(x,y))
results = minTemps.collect();

for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

ITE00100554	90.14F
EZE00100082	90.14F


In [66]:
# WORK COUNT #2
inputs = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/book.txt")
print(type(inputs),inputs.count())
words = inputs.flatMap(lambda x: x.split())
print(type(words),words.count())
wordCount = words.countByValue() # count by value on a rdd returned dictionary
print(type(wordCount),wordCount)
for key,value in wordCount.items(): # all old good python ,can be better by type conversion
    print(key,value)  

(<class 'pyspark.rdd.RDD'>, 926)
(<class 'pyspark.rdd.PipelinedRDD'>, 46249)
(<type 'collections.defaultdict'>, defaultdict(<type 'int'>, {u'considered.': 1, u'considered?': 1, u'Does': 2, u'ocean,': 1, u'Dartmouth,': 1, u'hanging': 1, u'payoff': 2, u'looking': 9, u'entirely\ufffd': 1, u'LAST': 1, u'Discarding': 1, u'oceans': 1, u'advice.': 1, u'advice,': 7, u'ever-improving': 1, u'writing,': 1, u'basics': 2, u'job),': 1, u'TELLING': 1, u'AdSense,': 1, u'non-compete': 2, u'Conventional': 1, u'27,': 1, u'second': 3, u'sales?': 1, u'Kane,': 2, u'Kane.': 1, u'(\ufffdObamacare\ufffd,)': 1, u'sales.': 6, u'sales,': 9, u'high-dollar': 1, u'designing': 2, u'"moderately': 1, u'resignation,': 1, u'resignation.': 1, u'leaning': 2, u'here': 11, u'careers.': 1, u'Rumsey,': 1, u'affiliated': 1, u'SDK,': 1, u'reasons:': 1, u'separation': 1, u'kids': 5, u'reports': 2, u'Choosing': 1, u'NOT': 2, u'symmetrical': 1, u'interesting?': 1, u'criticism': 1, u'Three': 2, u'replace': 4, u'brought': 1, u'remnan

(u'considered.', 1)
(u'considered?', 1)
(u'Does', 2)
(u'ocean,', 1)
(u'Dartmouth,', 1)
(u'hanging', 1)
(u'payoff', 2)
(u'looking', 9)
(u'entirely\ufffd', 1)
(u'LAST', 1)
(u'Discarding', 1)
(u'oceans', 1)
(u'advice.', 1)
(u'advice,', 7)
(u'ever-improving', 1)
(u'writing,', 1)
(u'basics', 2)
(u'job),', 1)
(u'TELLING', 1)
(u'AdSense,', 1)
(u'non-compete', 2)
(u'Conventional', 1)
(u'27,', 1)
(u'second', 3)
(u'sales?', 1)
(u'Kane,', 2)
(u'Kane.', 1)
(u'(\ufffdObamacare\ufffd,)', 1)
(u'sales.', 6)
(u'sales,', 9)
(u'high-dollar', 1)
(u'designing', 2)
(u'"moderately', 1)
(u'resignation,', 1)
(u'resignation.', 1)
(u'leaning', 2)
(u'here', 11)
(u'careers.', 1)
(u'Rumsey,', 1)
(u'affiliated', 1)
(u'SDK,', 1)
(u'reasons:', 1)
(u'separation', 1)
(u'kids', 5)
(u'reports', 2)
(u'Choosing', 1)
(u'NOT', 2)
(u'symmetrical', 1)
(u'interesting?', 1)
(u'criticism', 1)
(u'Three', 2)
(u'replace', 4)
(u'brought', 1)
(u'remnant', 1)
(u'anywhere.', 1)
(u'stern', 1)
(u'"The', 4)
(u'cheating', 1)
(u'revenue,', 2)

(u'puts', 2)
(u'software', 33)
(u'three', 6)
(u'assess', 1)
(u'ads.', 10)
(u'housing', 2)
(u'overestimate', 2)
(u'trigger', 1)
(u'IMDb', 1)
(u'place,', 2)
(u'indoctrinated', 1)
(u"We'll", 4)
(u'road.', 1)
(u'delivery', 1)
(u'construction', 2)
(u'letter.', 2)
(u'delivers', 1)
(u'image-focused,', 1)
(u'count', 3)
(u'Telescope', 1)
(u'places', 3)
(u'haven\ufffdt', 2)
(u'hours\ufffd,', 1)
(u'excitement', 1)
(u'spot.', 1)
(u'Requirments', 1)
(u'problem', 16)
(u'private.', 1)
(u'Definition', 1)
(u'recognize', 1)
(u'contribute', 2)
(u'compared', 3)
(u'variety', 1)
(u'fired.', 1)
(u'effective.', 4)
(u'incubators,', 1)
(u'Else', 1)
(u'reach,', 1)
(u'reach.', 3)
(u'Understand', 2)
(u'details', 3)
(u'effective;', 1)
(u'in?', 1)
(u'again.', 6)
(u'again,', 5)
(u'worker', 2)
(u'in,', 6)
(u'chance', 12)
(u'outdated', 1)
(u'Page', 2)
(u'exposure', 2)
(u'searches', 2)
(u'detail,', 1)
(u"website's", 2)
(u'lasted', 3)
(u'rule', 1)
(u'trial.', 1)
(u'searched', 4)
(u'Happen', 1)
(u'saves', 1)
(u'desirable'

(u'purchases', 1)
(u'compare', 3)
(u'scary.', 1)
(u'scary,', 1)
(u'share', 9)
(u'contradicts', 1)
(u'minimum', 2)
(u'numbers', 3)
(u'purchased', 3)
(u'hours.', 1)
(u'needs', 22)
(u'drained', 1)
(u'technology', 7)
(u'eat,', 1)
(u'number.', 1)
(u'number,', 3)
(u'List', 1)
(u'painful.', 1)
(u'questioning', 1)
(u'coming', 17)
(u'purchase.', 2)
(u'response', 4)
(u'purchase,', 1)
(u'Plans', 1)
(u'overstated.', 1)
(u'responsibility', 1)
(u'playing', 1)
(u'need.', 3)
(u'What', 38)
(u'Often,', 3)
(u'more,', 5)
(u'through', 55)
(u'steady,', 1)
(u'steady.', 1)
(u'suffer', 1)
(u'Texas', 1)
(u'22', 1)
(u'late', 1)
(u'2,', 1)
(u'recommends,', 1)
(u'crawls', 1)
(u'hurting', 1)
(u'someone', 57)
(u'seeking', 6)
(u'Realistic', 1)
(u'viewers', 1)
(u'easily', 7)
(u'deteriorates', 1)
(u'harm', 1)
(u'everyone', 2)
(u'mental', 4)
(u'house', 7)
(u'energy', 7)
(u'hard', 39)
(u'idea', 44)
(u'Build', 2)
(u'decision-maker,', 1)
(u'Aaron\ufffds', 1)
(u'insurance', 14)
(u'funding', 1)
(u'projected', 2)
(u'computer,

In [67]:
# if you input has unicode- which it ha s'u or utf-8 we can do a quick clean up to ascii
for word,count in wordCount.items():
    cleanWord =  word.encode('ascii','ignore')
    if(cleanWord):
        #print("the word is : %s,the count is: %s" %(cleanWord.decode(),str(count)))
        print("%s,%s" %(cleanWord.decode(),str(count)))

considered.,1
considered?,1
Does,2
ocean,,1
Dartmouth,,1
hanging,1
payoff,2
looking,9
entirely,1
LAST,1
Discarding,1
oceans,1
advice.,1
advice,,7
ever-improving,1
writing,,1
basics,2
job),,1
TELLING,1
AdSense,,1
non-compete,2
Conventional,1
27,,1
second,3
sales?,1
Kane,,2
Kane.,1
(Obamacare,),1
sales.,6
sales,,9
high-dollar,1
designing,2
"moderately,1
resignation,,1
resignation.,1
leaning,2
here,11
careers.,1
Rumsey,,1
affiliated,1
SDK,,1
reasons:,1
separation,1
kids,5
reports,2
Choosing,1
NOT,2
symmetrical,1
interesting?,1
criticism,1
Three,2
replace,4
brought,1
remnant,1
anywhere.,1
stern,1
"The,4
cheating,1
revenue,,2
concept!,1
revenue.,4
catchy,1
revenue!,1
therefore,2
strike,1
until,15
show,,2
paperwork,2
relax,2
successful,14
brings,3
tying,1
hold,1
circumstances,1
pursue,1
liability,,1
placements.,1
marketer,,1
placements,,1
example,3
minimize.,1
household,1
Reality,1
caution,2
reviewing,1
want,99
minimum,,4
otherwise,,1
minimum.,3
medication?,1
statistics,,1
DPI,1
around.,1
ar

finalized.,1
ducks,1
E-mail,1
owes,1
seen,2
clouds:,1
Stock,1
normal,,1
So,,14
rules,4
U.S.,,1
Entrepreneurs,1
levels,2
necessity,1
NAMING,1
governments.,1
recent,2
Limited,2
Didn't,1
yet.,3
person,16
dollars,1
capture.,1
Quality,1
project.,1
Shelton,1
"their,1
point,,13
Labor,2
abbreviation,1
demand,,3
demand.,1
unto,1
readers,4
individual),1
emails,1
advertisements,2
individual.,2
parents,2
location,4
billable,2
relevance,1
flowchart,2
surprised,5
Both,2
Tracking,1
emergency,2
demands,1
Of,3
couple,4
instructors,1
1099,1
aerospace,2
"call,1
Or,3
projects,2
formal,1
ON,2
submission,,1
OK,6
OF,16
individuals,3
Anywhere,,1
spins,1
literally.,1
COMPOUNDING,1
buy.,2
and/or,4
Frank,10
behave,2
healthcare.gov,1
answers.,1
And,,2
might.,1
Look,4
agreement,3
case,1
co-working,1
obscured,1
by,109
metrics,19
Workweek:,1
anything,18
deserves,1
stable,,1
Naming,1
into,75
appropriate,2
primarily,3
ACT,1
NEW,1
NET,1
span,1
spam,1
spending,15
simulator,3
submit,1
custom,1
suit,1
opens,1
graciously,1

Offline,1
commuting,4
profit,,2
profit.,4
Description,2
authoritative,1
generally,5
handed,1
digital,10
dies,1
felt,4
HOW,1
POTENTIAL,1
stick,,1
aware,4
weekend,3
derail,1
billion,2
Constant,2
assume,5
daily,3
premiums,4
Page:,1
laws,2
profits,3
shop,3
managed,4
excellent,1
Such,1
(LPO),,1
manages,1
car,),1
retention,1
depend,6
interact,,1
Exactly,1
editors,7
travel,,1
answered,1
else's.,1
off.,3
smartphone,1
half-assed,1
manage,,1
join,1
Where,2
governments,1
did,12
die,1
dig,3
item,4
editor,,1
Here's,7
(SBA),,1
attracting,1
software,,3
software.,1
unexpected,1
fearing,1
else,,5
dealing,5
else.,6
measurable,2
running.,1
else?,1
email-based,1
attainable,,1
anecdotes,1
statistic,1
covers,,1
suspect,1
Seriously,,1
filled,1
target.,1
ineffective,,1
"skyscraper,1
wait,2
box,2
up-front,1
shift,1
Founded,1
here.,2
Technology,2
genuinely,3
useful,8
merely,1
When,31
flexibility,2
licensing,1
wealth,1
sake,2
visit,8
review.,1
try.,1
Maintaining,2
"strongly,1
sharing,2
Orlando,4
Vietnam.,1
effor

essential,1
ABOUT,1
restricted,,1
understood,1
newsletter,1
Email,5
encouraging?,1
Fire,1
entrepreneur.,1
tends,1
prod,1
Times,1
requirements,,1
parents.,1
post-launch?,1
Prospective,1
age.,1
entrepreneurs,3
completing,1
range,3
complimentary,1
OPEN,1
question,8
long,40
Every,4
Amazon.,1
Amazon,,3
analyze,2
sections,2
Remarketing,1
repeatedly,1
filed,1
Load,1
section,,3
section?,1
highlight,1
etc.,4
called,9
number).,1
parks,,1
associated,6
worthwhile,1
all-consumed,1
theft.,1
Digging,1
generate!,1
generate.,1
tone,1
inequities,1
malicious,2
These,16
income,17
suffer.,1
users,6
retire,,1
problems,12
helping,2
generated,2
allowing,3
Making,5
unsold.,1
back?,2
reason,,3
ends,,1
happier,1
purposes.,3
purposes,,7
back.,3
back,,1
attaining,1
spent;,1
Management,,1
scroll,2
once,29
Based,1
Harmony,,1
Bureau,2
bedroom.,2
Department,1
Launching,2
note:,1
deal.,3
include,11
issue.,1
8-plus-hour-day,1
engineered,1
sorting,1
Ongoing,1
discussions.,1
Instagram.,1
Instagram,,1
notes,1
deals,2
dealt

In [80]:
# IMPROVING THE WORDCOUNT SCRIPT WITH REGULAR EXPRESSIONS
import re
def normalizeWords(text):
    return re.compile(r'\W+',re.UNICODE).split(text.lower())
# regex engine splits words and removed other characters ,handles unicode & then
# converts that word to lower case

input = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/book.txt")
#words = input.flatMap(lambda x: x.split())
words = input.flatMap(normalizeWords)
print (type(words))
wordCounts = words.countByValue()
print (type(wordCounts))

for word, count in wordCounts.items():
    cleanWord = word.encode('ascii', 'ignore')
    if (cleanWord):
        print(cleanWord.decode() + " " + str(count))

<class 'pyspark.rdd.PipelinedRDD'>
<type 'collections.defaultdict'>
limited 12
similarity 1
hats 1
saves 1
desirable 1
enforceable 1
four 1
immersive 1
sleep 5
preface 2
saved 6
payoff 2
relationships 1
someplace 7
violate 1
crises 1
disability 2
extrapolate 1
swat 1
under 8
worth 39
risk 18
commerce 1
blanket 2
oceans 4
stolen 1
every 32
updates 3
lecture 1
affect 9
reclaiming 1
electronics 1
vast 2
school 3
basics 12
called 9
skills 17
cause 3
venue 1
integrated 2
unrelated 1
math 3
inaction 1
heading 1
whirlwind 1
enjoy 13
force 1
disciplines 2
consistent 1
hanging 2
estimates 3
direct 13
horn 2
surrounding 3
electricians 1
street 2
estimated 2
alt 1
even 104
established 2
errors 1
lacked 1
decisions 5
selected 1
near 5
asia 1
bonuses 2
above 5
designing 4
tactful 1
new 153
net 13
increasing 1
ever 10
told 6
hobby 2
specialist 1
evolving 1
behavior 2
mailings 1
never 27
disposable 1
here 29
hundreds 5
refinement 2
protection 7
china 4
pursuit 1
active 4
path 19
100 10
cardboard 1
10

effects 3
honesty 2
gates 1
outdoor 1
212 1
being 32
money 86
shock 1
fueling 2
physically 2
risking 1
reputable 2
cease 1
aspect 3
touch 3
polish 3
speed 1
incurring 1
weekly 1
refills 1
presumption 1
gather 2
application 2
thinking 10
trigger 3
seems 5
except 2
desktop 1
blog 10
parts 4
lets 5
setting 7
pile 2
gloating 2
thoughts 2
republic 1
unfinished 1
careers 3
real 31
moz 3
starving 1
negotiating 1
around 20
rules 4
handling 2
discontinue 1
admitting 1
early 11
outsource 5
traffic 33
venues 1
mom 1
schedules 2
world 24
accepted 3
vague 1
furniture 1
fortune 1
identifying 4
suppliers 2
serves 2
server 1
sanity 1
benefit 6
grants 2
t 301
always 19
served 2
passionate 1
reaches 1
burdens 1
maximize 9
simulator 3
competition 5
calling 2
helpcenter 1
manufacturers 1
respect 1
images 7
diving 2
bigger 5
bizarre 1
users 6
provided 4
slice 2
dimensions 1
frees 1
ability 15
overly 1
legal 3
conservative 1
dealership 1
exit 2
statements 3
provider 2
ulterior 1
allowing 3
quitting 10
refer

caffeine 1
h2 1
h1 1
washington 1
commitment 2
billions 1
due 6
assess 1
guard 1
weather 1
promise 1
actions 5
national 1
artists 1
kick 1
strategy 7
initially 6
utility 1
behind 3
permits 3
networking 2
additional 7
commission 3
prospects 4
housing 2
apps 2
secret 1
noticed 1
overestimate 2
intention 1
accountant 19
dropping 2
biggest 2
indoctrinated 1
touted 1
tricky 1
receipt 1
alexa 1
funds 2
brand 1
rumsey 1
but 242
delivery 1
reminds 1
lowered 1
gain 4
automated 12
delivers 1
editing 1
eat 4
he 9
count 4
also 91
made 13
safety 7
places 4
whether 21
wish 2
j 1
variations 1
hosting 6
signed 4
excitement 1
placed 1
below 3
frequency 1
stories 3
invest 11
distribution 6
pm 2
minutes 4
tricks 1
ring 3
dots 1
marketplace 7
replying 1
intimidating 2
recognize 1
universal 2
penny 2
openforum 1
consumed 1
highlighted 1
education 2
naming 2
authority 4
inc 1
avoided 1
variety 1
46 3
47 1
fraudulent 1
virtual 3
lately 2
forgivable 1
details 5
normal 1
welcomes 1
mass 5
out 161
clicks 3
disp

In [73]:
normalizeWords("hello heLLO HELLO")

['hello', 'hello', 'hello']

In [134]:
# sorting word count
import re
def normalizeWords(text):
    return re.compile(r'\W+',re.UNICODE).split(text.lower())
# regex engine splits words and removed other characters ,handles unicode & then
# converts that word to lower case

input = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/book.txt")
#words = input.flatMap(lambda x: x.split())
words = input.flatMap(normalizeWords)
print (type(words),words.take(3))
wordCounts = words.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
print(type(wordCounts),wordCounts.take(3))
print(wordCounts.map(lambda x : (x[1],x[0])).take(4)) # switched key s the count NOW
wordCountsSorted = wordCounts.map(lambda x : (x[1],x[0])).sortByKey(ascending=False)
print(type(wordCountsSorted),wordCountsSorted.take(3))
results = wordCountsSorted.collect()
print(type(results))

for result in results:
    count = str(result[0])
    word = result[1].encode('ascii','ignore') # ignore errors
    if (word):
        print(word.decode()+ ":\t\t" +count )



(<class 'pyspark.rdd.PipelinedRDD'>, [u'self', u'employment', u'building'])
(<class 'pyspark.rdd.PipelinedRDD'>, [(u'', 772), (u'limited', 12), (u'similarity', 1)])
[(772, u''), (12, u'limited'), (1, u'similarity'), (1, u'hats')]
(<class 'pyspark.rdd.PipelinedRDD'>, [(1878, u'you'), (1828, u'to'), (1420, u'your')])
<type 'list'>
you:		1878
to:		1828
your:		1420
the:		1292
a:		1191
of:		970
and:		934
that:		747
it:		649
in:		616
is:		560
for:		537
on:		428
are:		424
if:		411
s:		391
i:		387
business:		383
can:		376
be:		369
as:		343
have:		321
with:		315
t:		301
this:		280
or:		278
time:		255
but:		242
they:		234
will:		231
what:		229
at:		220
my:		215
re:		214
do:		207
not:		203
about:		202
more:		200
product:		182
an:		178
up:		177
need:		174
from:		166
them:		166
how:		163
there:		162
out:		161
new:		153
people:		145
work:		144
so:		143
just:		142
own:		140
all:		137
don:		133
get:		123
customers:		123
want:		122
company:		122
by:		122
their:		122
some:		121
ll:		114
self:		111
websi

describing:		2
integration:		2
behave:		2
ries:		2
feelings:		2
wealth:		2
consultation:		2
analyze:		2
sections:		2
manufacture:		2
deductions:		2
defined:		2
influence:		2
engineering:		2
whom:		2
inequities:		2
malicious:		2
gospel:		2
matching:		2
occasional:		2
definitely:		2
housekeepers:		2
depicting:		2
tank:		2
bankers:		2
fresh:		2
scroll:		2
issues:		2
young:		2
wave:		2
tradeoffs:		2
forbes:		2
deals:		2
disclaimer:		2
waking:		2
waiting:		2
experiment:		2
flowing:		2
body:		2
shelton:		2
explore:		2
consideration:		2
menu:		2
bookkeeping:		2
involves:		2
scales:		2
win:		2
crazy:		2
private:		2
moonlighting:		2
fee:		2
lens:		2
determining:		2
pitfalls:		2
iii:		2
subscription:		2
announcing:		2
challenge:		2
refine:		2
employs:		2
invoices:		2
tab:		2
smartphone:		2
factored:		2
light:		2
healthcare:		2
adjust:		2
poor:		2
produced:		2
looks:		2
chosen:		2
estimating:		2
2013:		2
crash:		2
hands:		2
glasses:		2
globe:		2
staying:		2
society:		2
chaos:		2
clarify:		2
x:		2

In [143]:
# RDD SORT
rdd = sc.parallelize([1,5,3,4,2])
srted = rdd.sortBy(lambda x:x ,ascending=False).collect() # .collect generates list ,countByValue a dict
print (type(srted),srted)

lst = [('hello', 772), ('limited', 12), ('similarity', 123),('dissimilarity', 120)]

newrdd =  sc.parallelize(lst)
#srtedKey = rdd.sortByKey()
print (type(newrdd),newrdd.collect())

swap = newrdd.map(lambda x:(x[1],x[0]))
print(swap.collect())

sortd = swap.sortByKey(ascending=False)
print(sortd).collect()

(<type 'list'>, [5, 4, 3, 2, 1])
(<class 'pyspark.rdd.RDD'>, [('hello', 772), ('limited', 12), ('similarity', 123), ('dissimilarity', 120)])
[(772, 'hello'), (12, 'limited'), (123, 'similarity'), (120, 'dissimilarity')]
[(772, 'hello'), (123, 'similarity'), (120, 'dissimilarity'), (12, 'limited')]


In [172]:
# FIND THE ORDERS BY COSTOMER ID
lines = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/customer-orders.csv")

In [181]:
print("count : ",lines.count(),lines.take(4))

('count : ', 10000, [u'44,8602,37.19', u'35,5368,65.89', u'2,3391,40.64', u'47,6694,14.98'])


In [191]:
# write a parsing def to pull out cust id and orders
def parseLine(line):
    fields = line.split(',')
    customerId = int(fields[0])
    orders = float(fields[2])
    return(customerId,orders)
    print(customerId,orders)

In [192]:
parsedRDD =  lines.map(parseLine) # instead of doing split and field set in lambda 
# I am doing it in a def

In [193]:
parsedRDD.take(4)

[(44, 37.19), (35, 65.89), (2, 40.64), (47, 14.98)]

In [194]:
# Now find total order by customer
orderByCust = parsedRDD.reduceByKey(lambda x,y:x+y)

In [195]:
orderByCust.take(10)

[(0, 5524.949999999998),
 (1, 4958.600000000001),
 (2, 5994.59),
 (3, 4659.63),
 (4, 4815.050000000002),
 (5, 4561.069999999999),
 (6, 5397.879999999998),
 (7, 4755.070000000001),
 (8, 5517.240000000001),
 (9, 5322.649999999999)]

In [196]:
# NOW SORT BY AMOUnT
swapKey = orderByCust.map(lambda x:(x[1],x[0]))
swapKey.take(10)

[(5524.949999999998, 0),
 (4958.600000000001, 1),
 (5994.59, 2),
 (4659.63, 3),
 (4815.050000000002, 4),
 (4561.069999999999, 5),
 (5397.879999999998, 6),
 (4755.070000000001, 7),
 (5517.240000000001, 8),
 (5322.649999999999, 9)]

In [201]:
sortedAmoutnt = swapKey.sortByKey(ascending=False).collect()

for x in sortedAmoutnt:
    print(x)

(6375.449999999997, 68)
(6206.199999999999, 73)
(6193.109999999999, 39)
(6065.389999999999, 54)
(5995.660000000003, 71)
(5994.59, 2)
(5977.189999999995, 97)
(5963.109999999999, 46)
(5696.840000000003, 42)
(5642.89, 59)
(5637.62, 41)
(5524.949999999998, 0)
(5517.240000000001, 8)
(5503.43, 85)
(5497.479999999998, 61)
(5496.050000000004, 32)
(5437.7300000000005, 58)
(5415.150000000001, 63)
(5413.510000000001, 15)
(5397.879999999998, 6)
(5379.280000000002, 92)
(5368.83, 43)
(5368.249999999999, 70)
(5337.44, 72)
(5330.8, 34)
(5322.649999999999, 9)
(5298.090000000002, 55)
(5290.409999999998, 90)
(5288.689999999996, 64)
(5265.750000000001, 93)
(5259.920000000003, 24)
(5254.659999999998, 33)
(5253.3200000000015, 62)
(5250.4, 26)
(5245.059999999999, 52)
(5206.4, 87)
(5186.429999999999, 40)
(5155.419999999999, 35)
(5152.290000000002, 11)
(5140.3499999999985, 65)
(5123.010000000001, 69)
(5112.709999999999, 81)
(5059.4299999999985, 19)
(5057.610000000001, 25)
(5040.709999999999, 60)
(5032.67999999

In [2]:
# we have covered RDD sort,map,reduce (aggregation) # here is how we join
store_sale = [('2003',45),('2000',65),('2003',25),('2000',15),('3000',45)]
store_description = [('2003','High street Metro'),('2000','Park way Express'),('3000','jj metro')]
store_type = [('2003','new'),('2000','old'),('3000','old')]

In [3]:
store_saleRDD = sc.parallelize(store_sale)
print(store_saleRDD.collect())
store_descriptionRDD = sc.parallelize(store_description)
print(store_descriptionRDD.collect())
store_typeRDD = sc.parallelize(store_type)
print(store_typeRDD.collect())

[('2003', 45), ('2000', 65), ('2003', 25), ('2000', 15), ('3000', 45)]
[('2003', 'High street Metro'), ('2000', 'Park way Express'), ('3000', 'jj metro')]
[('2003', 'new'), ('2000', 'old'), ('3000', 'old')]


In [4]:
joinedRDD =store_saleRDD.join(store_descriptionRDD).join(store_typeRDD).collect()
joinedRDD

[('2003', ((45, 'High street Metro'), 'new')),
 ('2003', ((25, 'High street Metro'), 'new')),
 ('2000', ((65, 'Park way Express'), 'old')),
 ('2000', ((15, 'Park way Express'), 'old')),
 ('3000', ((45, 'jj metro'), 'old'))]

In [5]:
for i in joinedRDD:
    print("store : ", i[0], "sales :", i[1][0][0], "addr :", i[1][0][1],"type: ",i[1][1][0:])

('store : ', '2003', 'sales :', 45, 'addr :', 'High street Metro', 'type: ', 'new')
('store : ', '2003', 'sales :', 25, 'addr :', 'High street Metro', 'type: ', 'new')
('store : ', '2000', 'sales :', 65, 'addr :', 'Park way Express', 'type: ', 'old')
('store : ', '2000', 'sales :', 15, 'addr :', 'Park way Express', 'type: ', 'old')
('store : ', '3000', 'sales :', 45, 'addr :', 'jj metro', 'type: ', 'old')


In [6]:
store_saleRDD.collect()

[('2003', 45), ('2000', 65), ('2003', 25), ('2000', 15), ('3000', 45)]

In [8]:
storeDetails = [('2003','new','2017'),('2000','old','1995'),('3000','old','1999')]
storeDetailsRDD = sc.parallelize(storeDetails)

In [17]:
# sort on different fields and not key
srtd = storeDetailsRDD.sortBy(lambda x:x[2][1],ascending=False)
srtd.collect()

[('2000', 'old', '1995'), ('3000', 'old', '1999'), ('2003', 'new', '2017')]

In [18]:
srtd = storeDetailsRDD.sortBy(lambda x:x[2],ascending=False)
srtd.collect()

[('2003', 'new', '2017'), ('3000', 'old', '1999'), ('2000', 'old', '1995')]

In [19]:
storeDetailsRDD.join?

In [15]:
store_saleDF = store_saleRDD.toDF()
print(type(store_saleDF), store_saleDF.show())

+----+---+
|  _1| _2|
+----+---+
|2003| 45|
|2000| 65|
|2003| 25|
|2000| 15|
|3000| 45|
+----+---+

(<class 'pyspark.sql.dataframe.DataFrame'>, None)


In [90]:
store_saleDF = sqlContext.createDataFrame(store_saleRDD,schema=['retail','sale'])
print(type(store_saleDF), store_saleDF.show())

+------+----+
|retail|sale|
+------+----+
|  2003|  45|
|  2000|  65|
|  2003|  25|
|  2000|  15|
|  3000|  45|
+------+----+

(<class 'pyspark.sql.dataframe.DataFrame'>, None)


In [93]:
soredDF=store_saleDF.sort('sale','retail')
soredDF.show()

+------+----+
|retail|sale|
+------+----+
|  2000|  15|
|  2003|  25|
|  2003|  45|
|  3000|  45|
|  2000|  65|
+------+----+



In [96]:
soredDF=store_saleDF.sort(store_saleDF.retail.desc(),store_saleDF.sale.asc())
soredDF.show()

+------+----+
|retail|sale|
+------+----+
|  3000|  45|
|  2003|  25|
|  2003|  45|
|  2000|  15|
|  2000|  65|
+------+----+



In [21]:
# FINDING THE MOST POPULAR MOVIE
#1 load movies
#2 parse/split
#3 count movies and then order by desc

In [64]:
#1
lines = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/ml-100k/u.data")

In [97]:
print(lines.count(),lines.take(2))
def extractMovies(line):
    fields =  line.split()
    movieId = int(fields[1])
    return(movieId)


moviesRDD =  lines.map(extractMovies)
print(moviesRDD.take(2))

keyValRDD = moviesRDD.map(lambda x:(x,1)) # see# 2 lambda way
print(keyValRDD.take(3))

movieCount = keyValRDD.reduceByKey(lambda x,y:(x+y))
print(movieCount.take(3))

flipRDD = movieCount.map(lambda x: (x[1],x[0]) )
print(flipRDD.take(3))

sortedMovies=flipRDD.sortByKey( ascending=False)
print(sortedMovies.take(5)) # movie id 50 appears 583 times higest

(100000, [u'196\t242\t3\t881250949', u'186\t302\t3\t891717742'])
[242, 302]
[(242, 1), (302, 1), (377, 1)]
[(1, 452), (2, 131), (3, 90)]
[(452, 1), (131, 2), (90, 3)]
[(583, 50), (509, 258), (508, 100), (507, 181), (485, 294)]


In [98]:
# OR  2 lambda way
print(lines.take(2))
movies = lines.map(lambda x:(int(x.split()[1]),1)) # split and movied id in same function
movies.take(3)

[u'196\t242\t3\t881250949', u'186\t302\t3\t891717742']


[(242, 1), (302, 1), (377, 1)]

In [99]:
#POPULAR MOVIES WITH BROADCAST A SMALL TABLE
#THis is the process in which Spark uses python pbjects in mappers and reducers across all nodes

In [126]:
def loadMovieNames():
    movieNames = {}
    with open("ml-100k/u.item") as dfile:
        for line in dfile:
            fields = line.split("|")
            movieId = fields[0]
            movieNames[int(movieId)] = fields[1]
            #print(movieNames[1])
        return movieNames

    

{1: 'Toy Story (1995)',
 2: 'GoldenEye (1995)',
 3: 'Four Rooms (1995)',
 4: 'Get Shorty (1995)',...

In [142]:
nameDict = sc.broadcast(loadMovieNames())
print(nameDict.value[22]) # sampel id 22 passed ,you get value fro dict
lines = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/ml-100k/u.data")
movies = lines.map(lambda x: (int(x.split()[1]),1))
print(movies.take(3))
movieCounts = movies.reduceByKey(lambda x,y:x+y)
print(movieCounts.take(3))
flipped = movieCounts.map(lambda x: (x[1],x[0]))
print(flipped.take(3))
sortedMovies = flipped.sortByKey(ascending=False)
print("TOP 5 MOVIES : ",sortedMovies.take(5)) 

Braveheart (1995)
[(242, 1), (302, 1), (377, 1)]
[(1, 452), (2, 131), (3, 90)]
[(452, 1), (131, 2), (90, 3)]
('TOP 5 MOVIES : ', [(583, 50), (509, 258), (508, 100), (507, 181), (485, 294)])


In [141]:
sortedMoviesWithNames = sortedMovies.map(lambda (count, movie) : (nameDict.value[movie], count))
print(sortedMoviesWithNames.take(5))
results = sortedMoviesWithNames.collect()
#for i in results:
#    print(i)

[('Star Wars (1977)', 583), ('Contact (1997)', 509), ('Fargo (1996)', 508), ('Return of the Jedi (1983)', 507), ('Liar Liar (1997)', 485)]


In [145]:
sortedMoviesWithNames = sortedMovies.map(lambda (count,movieid) : (nameDict.value[movieid],count) )
sortedMoviesWithNames.take(5)

[('Star Wars (1977)', 583),
 ('Contact (1997)', 509),
 ('Fargo (1996)', 508),
 ('Return of the Jedi (1983)', 507),
 ('Liar Liar (1997)', 485)]

In [166]:
# MOST POPULAR SUPER HERO SOCIAL GRAPH
# who is the super hero that appears with most other characters?
names =  sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/Marvel-names.txt")
namesRdd = names.map(parseNames)
print(namesRdd.take(3))
names.count()

[(1, '24-HOUR MAN/EMMANUEL'), (2, '3-D MAN/CHARLES CHAN'), (3, '4-D MAN/MERCURIO')]


19428

In [153]:
lines =  sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/Marvel-graph.txt")
lines.count()

6589

In [155]:
def countCoOccurences(line):
    elements = line.split()
    print("type of elemsnts : ", type(elements))
    return (int(elements[0]), len(elements) - 1)

def parseNames(line):
    fields = line.split('\"')
    return (int(fields[0]), fields[1].encode("utf8"))

In [158]:
pairings = lines.map(countCoOccurences)
print(pairings.take(3)) # this means 5988 has 48 other heros appearing with it

[(5988, 48), (5989, 40), (5982, 42)]


In [160]:
totalFriendsByCharacter =  pairings.reduceByKey(lambda x,y:(x+y))
print(totalFriendsByCharacter.take(3)) 

[(1, 5), (2, 122), (3, 72)]


In [162]:
flipped = totalFriendsByCharacter.map(lambda x:(x[1],x[0]))
print(flipped.take(3)) 

[(5, 1), (122, 2), (72, 3)]


In [164]:
mostPopular = flipped.max()
print(mostPopular)

(1933, 859)


In [172]:
mostPopularName = namesRdd.lookup(mostPopular[1])[0]
print(mostPopularName)

CAPTAIN AMERICA


In [178]:
# redo
names =  sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/Marvel-names.txt")
print(names.take(1))
graph = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/Marvel-graph.txt")
print(graph.take(1))
# the first is s a name lookup rdd, the second has a super hero(0) and all other super heros he has co-appeared

[u'1 "24-HOUR MAN/EMMANUEL"']
[u'5988 748 1722 3752 4655 5743 1872 3413 5527 6368 6085 4319 4728 1636 2397 3364 4001 1614 1819 1585 732 2660 3952 2507 3891 2070 2239 2602 612 1352 5447 4548 1596 5488 1605 5517 11 479 2554 2043 17 865 4292 6312 473 534 1479 6375 4456 ']


In [228]:
def parseName(line):
    fields = line.split('\"') # " is sdelimiter and has to be escaped
    heroName = fields[1].encode("utf8") # tgis solbed unicoded text
    heroId = int(fields[0])
    
    return(heroId,heroName)

In [229]:
idAndName=names.map(parseName)
idAndName.take(2)

[(1, '24-HOUR MAN/EMMANUEL'), (2, '3-D MAN/CHARLES CHAN')]

In [230]:
def parseGraph(line):
    fields = line.split()
    firstHero = int(fields[0])
    countFrnds = len(fields)-1
    
    return(firstHero,countFrnds)

In [231]:
graphParsed=graph.map(parseGraph)
graphParsed.take(2)
print(graphParsed.take(2),graphParsed.count())
#graphParsed.countByValue()

([(5988, 48), (5989, 40)], 6589)


In [232]:
# adding all hero heros with same key
sumAllGRiends=graphParsed.reduceByKey(lambda x,y:(x+y))
print(sumAllGRiends.take(2),sumAllGRiends.count())

([(1, 5), (2, 122)], 6486)


In [235]:
# swap key before sort
swap =  sumAllGRiends.map(lambda x:(x[1],x[0]))
swap.take(2)
sortedRDD=swap.sortByKey(ascending=False)
print(sortedRDD.take(2))
maxFriends =  sortedRDD.max()
print(maxFriends)

[(1933, 859), (1741, 5306)]
(1933, 859)


In [240]:
mostPopularname = idAndName.lookup(maxFriends[1])
mostPopularname

['CAPTAIN AMERICA']

In [239]:
mostPopularname = idAndName.lookup(maxFriends[1])[0]
mostPopularname

'CAPTAIN AMERICA'

In [233]:
idAndName.lookup?

In [60]:
#Boilerplate stuff:
# The characters we wish to find the degree of separation between:
startCharacterID = 5000 #our gero
targetCharacterID = 2000  #ADAM 3,031 (who?)
# Our accumulator, used to signal when we find the target character during
# our BFS traversal.
hitCounter = sc.accumulator(0)

def convertToBFS(line):
    fields = line.split()
    heroID = int(fields[0])
    connections = []
    for connection in fields[1:]:
        connections.append(int(connection))

    color = 'WHITE'
    distance = 9999

    if (heroID == startCharacterID):
        color = 'GRAY'
        distance = 0

    return (heroID, (connections, distance, color))


def createStartingRdd():
    inputFile = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/Marvel-graph_dum.txt")
    return inputFile.map(convertToBFS)

def bfsMap(node): #(5000, ([748, 1722, 3752], 0, 'GRAY'))
    
    characterID = node[0]
    data = node[1]
    connections = data[0]
    distance = data[1]
    color = data[2]
    

    results = []

    #If this node needs to be expanded...
    if (color == 'GRAY'):
        for connection in connections:
            newCharacterID = connection
            newDistance = distance + 1
            newColor = 'GRAY'
            if (targetCharacterID == connection):
                hitCounter.add(1)

            newEntry = (newCharacterID, ([], newDistance, newColor))
            results.append(newEntry)

        #We've processed this node, so color it black
        color = 'BLACK'

    #Emit the input node so we don't lose it.
    results.append( (characterID, (connections, distance, color)) )
    return results

def bfsReduce(data1, data2): #x,y or i & i+1
    edges1 = data1[0]
    edges2 = data2[0]
    distance1 = data1[1]
    distance2 = data2[1]
    color1 = data1[2]
    color2 = data2[2]

    distance = 9999
    color = 'WHITE'
    edges = []
    

    # See if one is the original node with its connections.
    # If so preserve them.
    if (len(edges1) > 0):  # not applicable in our case
        edges.extend(edges1)
    if (len(edges2) > 0):  # ie we arepreserving larger connections 
        edges.extend(edges2)

    # Preserve minimum distance
    if (distance1 < distance):
        distance = distance1

    if (distance2 < distance):
        distance = distance2

    # Preserve darkest color
    if (color1 == 'WHITE' and (color2 == 'GRAY' or color2 == 'BLACK')):
        color = color2

    if (color1 == 'GRAY' and color2 == 'BLACK'):
        color = color2

    if (color2 == 'WHITE' and (color1 == 'GRAY' or color1 == 'BLACK')):
        color = color1

    if (color2 == 'GRAY' and color1 == 'BLACK'):
        color = color1

    return (edges, distance, color)


#Main program here:
iterationRdd = createStartingRdd()
print("first rdd: ",iterationRdd.collect())
print("----------------------")

for iteration in range(0, 10):
    print("Running BFS iteration# " + str(iteration+1))

    # Create new vertices as needed to darken or reduce distances in the
    # reduce stage. If we encounter the node we're looking for as a GRAY
    # node, increment our accumulator to signal that we're done.
    #print(iterationRdd.collect())
    mapped = iterationRdd.flatMap(bfsMap)
    print("mapper result: ",mapped.collect())
    print("-------------------")

    # Note that mapped.count() action here forces the RDD to be evaluated, and
    # that's the only reason our accumulator is actually updated.
    print("Processing " + str(mapped.count()) + " values.")

    if (hitCounter.value > 0):
        print("Hit the target character! From " + str(hitCounter.value) \
            + " different direction(s).")
        break

    # Reducer combines data for each character ID, preserving the darkest
    # color and shortest path.
    iterationRdd = mapped.reduceByKey(bfsReduce)
    print("REDUCER O/P :",iterationRdd.collect())


('first rdd: ', [(5000, ([748, 1722, 3752], 0, 'GRAY')), (1722, ([2010, 4264, 4446], 9999, 'WHITE')), (3752, ([4264, 595, 4447], 9999, 'WHITE')), (748, ([1165, 4264, 4361], 9999, 'WHITE')), (4264, ([5002, 1122], 9999, 'WHITE')), (4361, ([2000, 4363, 4088], 9999, 'WHITE'))])
----------------------
Running BFS iteration# 1
('mapper result: ', [(748, ([], 1, 'GRAY')), (1722, ([], 1, 'GRAY')), (3752, ([], 1, 'GRAY')), (5000, ([748, 1722, 3752], 0, 'BLACK')), (1722, ([2010, 4264, 4446], 9999, 'WHITE')), (3752, ([4264, 595, 4447], 9999, 'WHITE')), (748, ([1165, 4264, 4361], 9999, 'WHITE')), (4264, ([5002, 1122], 9999, 'WHITE')), (4361, ([2000, 4363, 4088], 9999, 'WHITE'))])
-------------------
Processing 9 values.
('REDUCER O/P :', [(3752, ([4264, 595, 4447], 1, 'GRAY')), (4361, ([2000, 4363, 4088], 9999, 'WHITE')), (748, ([1165, 4264, 4361], 1, 'GRAY')), (5000, ([748, 1722, 3752], 0, 'BLACK')), (1722, ([2010, 4264, 4446], 1, 'GRAY')), (4264, ([5002, 1122], 9999, 'WHITE'))])
Running BFS iter

In [36]:
# work page

In [55]:
def testred(x,y):
    edges1 = x[0]
    edges2 = y[0]
    
    return(edges1)

#print(mapped.collect())
iterationRdds = mapped.reduceByKey(testred)
iterationRdds.collect()

[(1122, ([], 3, 'GRAY')),
 (4264, ([5002, 1122], 2, 'BLACK')),
 (4361, ([2000, 4363, 4088], 2, 'BLACK')),
 (5002, ([], 3, 'GRAY')),
 (4363, ([], 3, 'GRAY')),
 (748, ([1165, 4264, 4361], 1, 'BLACK')),
 (1165, ([], 2, 'BLACK')),
 (2000, ([], 3, 'GRAY')),
 (3752, ([4264, 595, 4447], 1, 'BLACK')),
 (5000, ([748, 1722, 3752], 0, 'BLACK')),
 (595, ([], 2, 'BLACK')),
 (4088, ([], 3, 'GRAY')),
 (2010, ([], 2, 'BLACK')),
 (1722, ([2010, 4264, 4446], 1, 'BLACK')),
 (4446, ([], 2, 'BLACK')),
 (4447, ([], 2, 'BLACK'))]

In [33]:
mapped.collect() # takes ([], 3, 'GRAY') or ([2000, 4363, 4088], 2, 'BLACK')

[(5002, ([], 3, 'GRAY')),
 (1122, ([], 3, 'GRAY')),
 (4264, ([5002, 1122], 2, 'BLACK')),
 (2000, ([], 3, 'GRAY')),
 (4363, ([], 3, 'GRAY')),
 (4088, ([], 3, 'GRAY')),
 (4361, ([2000, 4363, 4088], 2, 'BLACK')),
 (3752, ([4264, 595, 4447], 1, 'BLACK')),
 (748, ([1165, 4264, 4361], 1, 'BLACK')),
 (1165, ([], 2, 'BLACK')),
 (5000, ([748, 1722, 3752], 0, 'BLACK')),
 (595, ([], 2, 'BLACK')),
 (2010, ([], 2, 'BLACK')),
 (1722, ([2010, 4264, 4446], 1, 'BLACK')),
 (4446, ([], 2, 'BLACK')),
 (4447, ([], 2, 'BLACK'))]

In [4]:
startUserId = 5000 
targetUserId = 2000 
# One accumulator, used to signal when we find the target character during
# our BFS traversal.
hitCounter = sc.accumulator(0)

def graphToBFS(line):
    fields = line.split()
    heroID = int(fields[0])
    connections = []
    for connection in fields[1:]:
        connections.append(int(connection))

    color = 'WHITE'
    distance = 9999

    if (heroID == startUserId):
        color = 'GRAY'
        distance = 0

    return (heroID, (connections, distance, color))


def createStartingRdd():
    inputFile = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/Marvel-graph_dum.txt")
    return inputFile.map(graphToBFS)

def bfsMap(node):
    
    characterID = node[0]
    data = node[1]
    connections = data[0]
    distance = data[1]
    color = data[2]
    

    results = []


    if (color == 'GRAY'):
        for connection in connections:
            newCharacterID = connection
            newDistance = distance + 1
            newColor = 'GRAY'
            if (targetUserId == connection):
                hitCounter.add(1)

            newEntry = (newCharacterID, ([], newDistance, newColor))
            results.append(newEntry)

        #We've processed this node, so color it black
        color = 'BLACK'

    #Emit the input node so we don't lose it.
    results.append( (characterID, (connections, distance, color)) )
    return results

def bfsReduce(data1, data2):
    edges1 = data1[0]
    edges2 = data2[0]
    distance1 = data1[1]
    distance2 = data2[1]
    color1 = data1[2]
    color2 = data2[2]

    distance = 9999
    color = 'WHITE'
    edges = []
    

    # See if one is the original node with its connections.
    # If so preserve them.
    if (len(edges1) > 0):  # not applicable in our case
        edges.extend(edges1)
    if (len(edges2) > 0):  # ie we arepreserving larger connections 
        edges.extend(edges2)

    # Preserve minimum distance
    if (distance1 < distance):
        distance = distance1

    if (distance2 < distance):
        distance = distance2

    # Preserve darkest color
    if (color1 == 'WHITE' and (color2 == 'GRAY' or color2 == 'BLACK')):
        color = color2

    if (color1 == 'GRAY' and color2 == 'BLACK'):
        color = color2

    if (color2 == 'WHITE' and (color1 == 'GRAY' or color1 == 'BLACK')):
        color = color1

    if (color2 == 'GRAY' and color1 == 'BLACK'):
        color = color1

    return (edges, distance, color)


iterationRdd = createStartingRdd()

for iteration in range(0, 10):
    print("Running BFS iteration# " + str(iteration+1))

    mapped = iterationRdd.flatMap(bfsMap)

    # mapped.count() action here forces the RDD to be evaluated
    print("Processing " + str(mapped.count()) + " values.")

    if (hitCounter.value > 0):
        print("Hit the target character! The Degree of Separation is : " + str(iteration+1))
        break

    # Reducer combines data for each character ID, preserving the darkest
    # color and shortest path.
    iterationRdd = mapped.reduceByKey(bfsReduce)


Running BFS iteration# 1
Processing 9 values.
Running BFS iteration# 2
Processing 16 values.
Running BFS iteration# 3
Processing 17 values.
Hit the target character! The Degree of Separation is : 3


In [2]:
    inputFile = sc.textFile("file:///home/vijay/DATA_SCIENCE/SC/SparkCourse/Marvel-graph_dum.txt")


In [3]:
inputFile.collect()

[u'5000 748 1722 3752',
 u'1722 2010 4264 4446',
 u'3752 4264 595 4447 2234',
 u'748 1165 4264 4361',
 u'4264 5002 1122',
 u'4361 2000 4363 4088']

In [80]:
srt1 = [5000,748,1722,3752,1722,2010,4264,4446]
srt2 = "1722 2010 4264 4446"
srt3 = "3752 4264 595 4447 2234"

In [81]:
rdd =  sc.parallelize(srt1)

In [82]:
rdd.collect()

[5000, 748, 1722, 3752, 1722, 2010, 4264, 4446]